# Impact of GDP per capita on employment in worldwide agriculture over the past 30 years  

Imports and set magics:

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go 
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from functools import reduce


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject

# import the API for WorldBank-data
from pandas_datareader import wb

# Read data

Here we are importing data using the API for WorldBank-data. We found the following data and data-codes using the Website of the WorldBank to provide us with different measures for countrys worldwide over the past decades:\
#
-Arable land in hectares --> *AG.LND.ARBL.HA*\
-Labor force --> *SL.TLF.TOTL.IN*\
-GDP per capita in USD --> *NY.GDP.PCAP.CD*\
-Employment in agriculture in % of total empoyment --> *SL.AGR.EMPL.ZS*

In [ ]:
# Setting up list of ISO-codes of countries where we are going to import data from using the API
countries_ISO_list = ['ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM', 'AUS', 'AUT', 
                      'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 
                      'BLZ', 'BMU', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 
                      'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 
                      'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 
                      'EGY', 'ERI', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'FRO', 'GAB', 'GBR', 
                      'GEO', 'GHA', 'GIB', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 
                      'GUY', 'HKG', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IMN', 'IND', 'IRL', 'IRN', 
                      'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 
                      'KIR', 'KNA', 'KOR', 'KWT', 'LBN', 'LBR', 'LBY', 'LCA', 'LIE', 'LKA', 'LSO', 
                      'LTU', 'LUX', 'LVA', 'MAC', 'MAF', 'MAR', 'MCO', 'MDA', 'MDG', 'MDV', 'MEX', 
                      'MKD', 'MLI', 'MLT', 'MMR', 'MNE', 'MNG', 'MOZ', 'MRT', 'MUS', 'MWI', 'MYS', 
                      'NAM', 'NCL', 'NER', 'NGA', 'NIC', 'NLD', 'NOR', 'NPL', 'NRU', 'NZL', 'OMN', 
                      'PAK', 'PAN', 'PER', 'PHL', 'PLW', 'PNG', 'POL', 'PRI', 'PRK', 'PRT', 'PRY', 
                      'PSE', 'PYF', 'QAT', 'ROU', 'RUS', 'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SLE', 
                      'SLV', 'SMR', 'SOM', 'SRB', 'SSD', 'STP', 'SUR', 'SVK', 'SVN', 'SWE', 'SWZ', 
                      'SXM', 'SYC', 'SYR', 'TCA', 'TCD', 'TGO', 'THA', 'TJK', 'TKM', 'TLS', 'TON', 
                      'TTO', 'TUN', 'TUR', 'TUV', 'TZA', 'UGA', 'UKR', 'URY', 'USA', 'UZB', 'VCT', 
                      'VEN', 'VGB', 'VIR', 'VNM', 'VUT', 'WSM', 'YEM', 'ZAF', 'ZMB', 'ZWE']

# Importing the different DataFrames from the WorldBank using the API from the lecture and resetting the index for each DataFrame
wb_arable_land = wb.download(indicator='AG.LND.ARBL.HA', country= countries_ISO_list, start=1989, end=2019)
wb_arable_land = wb_arable_land.reset_index()
wb_labor_force = wb.download(indicator='SL.TLF.TOTL.IN', country= countries_ISO_list, start=1989, end=2019)
wb_labor_force = wb_labor_force.reset_index()
wb_emp_ag = wb.download(indicator='SL.AGR.EMPL.ZS', country= countries_ISO_list, start=1989, end=2019)
wb_emp_ag = wb_emp_ag.reset_index()
wb_gdp_cap = wb.download(indicator='NY.GDP.PCAP.CD', country= countries_ISO_list, start=1989, end=2019)
wb_gdp_cap = wb_gdp_cap.reset_index()

# Clean and merge data 

In [ ]:
# changing the data-type of the 'year'-column to an integer
wb_arable_land.year = wb_arable_land.year.astype(int)
wb_labor_force.year = wb_labor_force.year.astype(int)
wb_emp_ag.year = wb_emp_ag.year.astype(int)
wb_gdp_cap.year = wb_gdp_cap.year.astype(int)

# Merging all the DataFrames into one big one
data_frames_list = [wb_arable_land, wb_labor_force, wb_emp_ag, wb_gdp_cap]
data_frames_combined = reduce(lambda left, right: pd.merge(left, right, on = ['country', 'year'], how = 'inner'), data_frames_list)

# Renaming the columns
namedict = {
            'AG.LND.ARBL.HA':'Arable_Land',
            'SL.TLF.TOTL.IN':'Labor_Force',
            'SL.AGR.EMPL.ZS':'Empl_in_agri_in_%_of_all_empl',
            'NY.GDP.PCAP.CD':'GDP_per_capita'}
data_frames_combined.rename(columns=namedict, inplace=True)

# Dropping all rows that contain empty values (NAN)
data_frames_combined.dropna(inplace=True)

# Rounding the floats in columns 'Empl_in_agri_in_%_of_all_empl' and 'GDP_per_capita' 
data_frames_combined['Empl_in_agri_in_%_of_all_empl'] = data_frames_combined['Empl_in_agri_in_%_of_all_empl'].round(2)
data_frames_combined['GDP_per_capita'] = data_frames_combined['GDP_per_capita'].round(2)

# Converting the values in the columns into their respective units for a better display
data_frames_combined['Empl_in_agri_in_%_of_all_empl'] = data_frames_combined['Empl_in_agri_in_%_of_all_empl'].apply(lambda x: '{:.2f}%'.format(x*1))
data_frames_combined['GDP_per_capita'] = data_frames_combined['GDP_per_capita'].apply(lambda x: '${:.2f}'.format(x))
data_frames_combined['Arable_Land'] = data_frames_combined['Arable_Land'].apply(lambda x: '{:.0f} ha'.format(x))
data_frames_combined.Labor_Force = data_frames_combined.Labor_Force.astype(int)

# Display the created DataFrame so far
data_frames_combined

## Exploring the individual data sets 

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [ ]:
countries_list = data_frames_combined['country'].unique().tolist()

def plot_gdp_per_capita_and_agri_empl(country):
    # Filter the data for the selected country
    country_data = data_frames_combined[data_frames_combined['country'] == country]

    # Create the plot
    fig = go.Figure()

    # Add GDP per capita trace (left y-axis)
    fig.add_trace(go.Scatter(x=country_data['year'], y=country_data['GDP_per_capita'],
                             mode='lines+markers', name='GDP per capita'))

    # Add percentage of people working in agriculture trace (right y-axis)
    fig.add_trace(go.Scatter(x=country_data['year'], y=country_data['Empl_in_agri_in_%_of_all_empl'],
                             mode='lines+markers', name='Employment in agriculture (%)',
                             yaxis='y2'))

    # Set the plot title and axis labels
    fig.update_layout(
        title=f'GDP per capita and employment in agriculture for {country}',
        xaxis_title='Year',
        yaxis_title='GDP per capita',
        yaxis2=dict(
            title='Employment in agriculture (%)',
            overlaying='y',
            side='right'
        )
    )

    # Show the plot
    fig.show()

# Create a dropdown menu to select a country
interact(plot_gdp_per_capita_and_agri_empl, country=widgets.Dropdown(options=countries_list, value=countries_list[0], description='Country:'))


DISCLAIMER: DUE TO THE AMOUNT OF DATA USED HERE IT MIGHT BE POSSIBLE THAT THE INTERACTIVE PLOT ABOVE DOES NOT AUTO-DISPLAY PROPERLY AND A COUNTRY MUST FIRST BE MANUALLY SELECTED!

EXPLANATION: 

# Analysis

We are now creating a subset of our DataFrame of the 10 countries with the largest area of arable land. In this Dataframe we will try to find a relationship between the influence of GDP per capita on the amount of arable land farmed per person working in agriculture. 

In [ ]:
# a. First change the units of the columns back to floats and integers to allow for calculation
data_frames_combined['Empl_in_agri_in_%_of_all_empl'] = data_frames_combined['Empl_in_agri_in_%_of_all_empl'].str.replace('%','').astype(float)
data_frames_combined['GDP_per_capita'] = data_frames_combined['GDP_per_capita'].str.replace('$','').astype(float)
data_frames_combined['Arable_Land'] = data_frames_combined['Arable_Land'].str.replace(' ha','').astype(int)

In [ ]:
# b. Creating a list of the 10 biggest countries by area of arable land 
df_sorted = data_frames_combined.sort_values('year', ascending=False)   # Sort big dataframe by year 
grouped = df_sorted.groupby('country')                                  # Group by country
latest_year = grouped.head(1)                                           # Keep rows for most recent year for each country
largest_countries = latest_year.nlargest(10, 'Arable_Land')             # Get the 10 largest in a DataFrame
country_column = largest_countries['country']                           # Get the country column as a Series object
top_ten_country_list = country_column.tolist()                          # Convert into a Python list

# Printing which countries we are going to keep 
print(f"The 10 biggest countries by area of arable land that will remain in the DataFrame are:\n {top_ten_country_list}")

# Cutting down the big WorldBank-DataFrame to the 10 countries with the largest area of arable land
data_frame_ten_biggest = data_frames_combined[data_frames_combined['country'].isin(top_ten_country_list)]
data_frame_ten_biggest = data_frame_ten_biggest.reset_index(drop=True)

# Displaying the new cut down Dataframe 
data_frame_ten_biggest

To get to the relationship between the GDP per capita and the area of arable land farmed by a single person working in the agriculture sector we will need to compute some new columns out of our original WorldBank-data due to the fact that not all the data was available in the way we would need it.

- We will have to compute the absolute amount of people working in the agriculture sector per country from the columns of the labor force and the percentage of people employed in agriculture. 

- We will also calculate a new column of the arable land divided by the people working in the agriculture sector. This then gives us an indicator on how much area of arable land one single worker can handle.

In [ ]:
# Create new column of absolute amount of people working in the agriculture sector
data_frame_ten_biggest['People_in_agri'] = (data_frame_ten_biggest['Labor_Force'] * (data_frame_ten_biggest['Empl_in_agri_in_%_of_all_empl'] / 100)).astype(int)

# Create a new column of the arable land divided by the people working in the agriculture sector
data_frame_ten_biggest['Land_per_agri_worker'] = data_frame_ten_biggest['Arable_Land'] / data_frame_ten_biggest['People_in_agri'] 

# Display the Dataframe  
data_frame_ten_biggest

**Interactive plot** :

In [ ]:
def plot_gdp_per_capita_and_land_per_agri_worker(country):

    country_data_new = data_frame_ten_biggest[data_frame_ten_biggest['country'] == country]

    x = country_data_new['GDP_per_capita']
    y = country_data_new['Land_per_agri_worker']
    m, b = np.polyfit(x, y, 1)
    line = m * x + b

    fig2 = go.Figure()

    fig2.add_trace(go.Scatter(
        x=country_data_new['GDP_per_capita'], 
        y=country_data_new['Land_per_agri_worker'], 
        mode='markers',
        marker=dict(
            size=10,
            color=country_data_new['year'], 
            colorscale='plasma', 
            opacity=0.8,
            colorbar=dict(
                title='Year'
            )
        ),
        text=country_data_new['year'],
        name=country
    ))

    fig2.add_trace(go.Scatter(x=x, y=line, mode='lines', name='Line of best fit'))

    fig2.update_layout(
        title=f'GDP per capita and land per agri worker {country}',
        xaxis_title='GDP per capita',
        yaxis_title='Land per agri worker',
        yaxis2=dict(
            title='Land per agri worker',
            overlaying='y',
            side='right'
        )
    )

    fig2.show() 


interact(plot_gdp_per_capita_and_land_per_agri_worker, country=widgets.Dropdown(options=top_ten_country_list, value=top_ten_country_list[3], description='Country:'))

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.